In [1]:
!pip install -q earthengine-api geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.0 MB/s eta 0:00:00


In [40]:
import time
import ee

In [41]:
# Authenticate (interactive OAuth)
ee.Authenticate()

True

In [42]:
# Initialize with Google Earth Engine Cloud Project ID
ee.Initialize(project='flood-gee-project')
print("Earth Engine initialized successfully")

Earth Engine initialized successfully


In [43]:
# 1. Defining only for a single state "Kerala"
STATE_NAME = 'Kerala'

# Load Administrative Boundaries for India (Level 1 = States)
admin_boundaries = ee.FeatureCollection("FAO/GAUL/2015/level1")

In [44]:
# Filter for the specific state
state = admin_boundaries.filter(ee.Filter.eq('ADM1_NAME', STATE_NAME)).first()

# Check if the state was found
if not state.getInfo():
    raise Exception(f"State '{STATE_NAME}' not found. Please check spelling and capitalization.")
else:
    print(f"Successfully selected the state of {STATE_NAME}.")

# Load JRC Global Surface Water data
gsw_occurrence = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select('occurrence')

# 2. Create a Classification Image
# Where occurrence >= 10, the value is 1 (flood). Everywhere else, it is 0 (no-flood).
class_band = ee.Image(0).where(gsw_occurrence.gte(10), 1).rename('label')

# 3. Combine Original Data with Class Labels
image_to_sample = gsw_occurrence.addBands(class_band)

# 4. Performing Stratified Sampling for the State
# We sample within the selected state's geometry.
# The scale can be smaller (more detailed) since the area is smaller.
dataset = image_to_sample.stratifiedSample(
    numPoints=1000,
    classBand='label',
    region=state.geometry(),
    scale=250,
    geometries=True,
    tileScale=4
)

print("Total sampled points:", dataset.size().getInfo())
print("Sample feature:", dataset.first().getInfo())

# 5. Export the Final Dataset
task = ee.batch.Export.table.toDrive(
    collection=dataset,
    description=f'{STATE_NAME}_Flood_NoFlood_GSW',
    fileFormat='CSV',
    folder='India_Flood_Dataset',
    selectors=['occurrence', 'label', '.geo']
)
task.start()
print("Export task started ✅. Check the 'Tasks' tab in the GEE Code Editor or your Google Drive.")

# Monitor task status
while task.active():
    print(f"Task running... (State: {task.status()['state']})")
    time.sleep(30)

final_status = task.status()
if final_status['state'] == 'COMPLETED':
    print("✅ Task completed successfully! Please check your Google Drive.")
else:
    error_message = final_status.get('error_message', 'No error message.')
    print(f"❌ Task failed or was cancelled: {error_message}")


Successfully selected the state of Kerala.
Total sampled points: 2000
Sample feature: {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [76.7800403962722, 10.709939396093986]}, 'id': '0', 'properties': {'label': 0, 'occurrence': 7}}
Export task started ✅. Check the 'Tasks' tab in the GEE Code Editor or your Google Drive.
Task running... (State: READY)
✅ Task completed successfully! Please check your Google Drive.
